<a href="https://colab.research.google.com/github/Aditya-y9/ApnaMakaan/blob/main/Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from shapely.geometry import Polygon

In [36]:
def centroid(room):
    x1 = room['position'][0]
    y1 = room['position'][1]
    x2 = room['position'][0] + room['size'][0]
    y2 = room['position'][1]
    x3 = room['position'][0] + room['size'][0]
    y3 = room['position'][1] + room['size'][1]
    x4 = room['position'][0]
    y4 = room['position'][1] + room['size'][1]
    vertices = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
    n = len(vertices)
    A = 0
    Cx = 0
    Cy = 0

    for i in range(n):
        x_i, y_i = vertices[i]
        x_ip1, y_ip1 = vertices[(i + 1) % n]  # i+1 modulo n to wrap around to the first vertex
        cross_product = (x_i * y_ip1) - (x_ip1 * y_i)
        A += cross_product
        Cx += (x_i + x_ip1) * cross_product
        Cy += (y_i + y_ip1) * cross_product

    A *= 0.5
    Cx /= (6 * A)
    Cy /= (6 * A)

    return (Cx, Cy)


Centroid: (46.0, 95.5)


In [28]:
# input like [{'name': 'By_laws', 'position": (60.8,
# 108.8), 'size': (19.2, 11.2)), {'name': 'Living Room', 'position': (12,
# 71), 'size': (68, 49)}, {'name': 'Passage', 'position': (0, 25), 'size': (12, 81)}, (name: Kitchen, position: (0, 51), size: (12, 25)), {'name': 'Bedroom1', 'position': (12, 12), 'size': (30, 30)}, ('name':
# 'Washroom', 'position': (32, 32), 'size': (7.5, 7)}]

def calculate_area_fitness(room):
  # input : {'name': 'Living Room', 'position': (12, 71), 'size': (68, 49)}
  x1 = room['position'][0]
  y1 = room['position'][1]
  x2 = room['position'][0] + room['size'][0]
  y2 = room['position'][1] + room['size'][1]
  coordinates = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
  polygon = Polygon(coordinates)
  area = polygon.area
  return area

In [41]:
import numpy as np

def irregularity(room_centroid, plot_vertices):
    # Calculate distances from room centroid to plot edges
    absolute_distances = []
    for vertex in plot_vertices:
        distance = np.sqrt((room_centroid[0] - vertex[0])**2 + (room_centroid[1] - vertex[1])**2)
        absolute_distances.append(distance)
    # Calculate the mean distance
    mean_distance = np.mean(absolute_distances)
    # Calculate the irregularity
    irregularity = 0
    for distance in absolute_distances:
        irregularity += np.abs(distance - mean_distance)
    return irregularity


In [43]:
import math

def calculate_distance_fitness(plot_centroid, figure_centroid, distance):
    plot_x, plot_y = plot_centroid
    figure_x, figure_y = figure_centroid

    # Distance penalty
    distance_penalty = distance / max_distance  # Normalize distance

    # Corner preference
    plot_width = max_plot_width  # Adjust as per your plot size
    plot_height = max_plot_height  # Adjust as per your plot size
    corner_preference = 1.0  # Maximum preference for corners

    # Calculate proximity to corners
    min_x_distance_to_corner = min(figure_x, plot_width - figure_x)
    min_y_distance_to_corner = min(figure_y, plot_height - figure_y)

    corner_penalty = 1.0 - (min_x_distance_to_corner + min_y_distance_to_corner) / (plot_width + plot_height)

    # Combine penalties with a smooth gradient
    fitness = (1 - corner_preference) * (1 - distance_penalty) + corner_preference * (1 - corner_penalty)

    return fitness

# Example usage:
plot_centroid = (5, 9)  # Example plot centroid coordinates
figure_centroid = (3, 5)  # Example living room figure centroid coordinates
distance_between_centroids = math.sqrt((plot_centroid[0] - figure_centroid[0])**2 + (plot_centroid[1] - figure_centroid[1])**2)

# Constants for normalization or specific plot dimensions
max_distance = 20  # Maximum expected distance between centroids
max_plot_width = 10.0  # Maximum width of the plot
max_plot_height = 10.0  # Maximum height of the plot

fitness_score = 0.5 - calculate_distance_fitness(plot_centroid, figure_centroid, distance_between_centroids)
print(f"Fitness score: {fitness_score}")


Fitness score: 0.09999999999999998


In [39]:
def calculate_fitness_gradient(distance):
    # Define a maximum reasonable distance for normalization
    max_distance = 20.0  # Adjust based on your scenario

    # Calculate fitness based on distance
    if distance <= max_distance:
        fitness = 1.0 - (distance / max_distance)
    else:
        fitness = 0.0  # If distance exceeds max_distance, fitness is 0

    return fitness

In [40]:
from shapely.geometry import Polygon, Point
from shapely.geometry.polygon import orient

def calculate_area(vertices):
    # Calculate area of polygon using shoelace formula
    n = len(vertices)
    area = 0.0
    for i in range(n):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % n]
        area += x1 * y2 - y1 * x2
    return abs(area) / 2.0

def calculate_boundary_overlap_penalty(room_vertices, plot_vertices):
    # Create Shapely Polygon objects
    room_polygon = Polygon(room_vertices)
    plot_polygon = Polygon(plot_vertices)

    # Calculate area of room and plot
    room_area = room_polygon.area
    plot_area = plot_polygon.area

    # Calculate intersection area
    intersection_area = room_polygon.intersection(plot_polygon).area

    # Check if the room is completely within the plot
    if room_area == intersection_area:
        boundary_overlap_penalty = 0.0  # Room is completely within the plot
    else:
        # Calculate area outside the plot
        area_outside_plot = room_area - intersection_area

        # Penalize proportional to area outside the plot
        boundary_overlap_penalty = area_outside_plot / room_area

    return boundary_overlap_penalty

In [42]:

def calculate_fitness(self, room_list):
  pass

In [ ]:
room_list = [{'name': 'Living Room', 'position': (12, 71), 'size': (68, 49)}]

In [29]:
calculate_area_fitness(room_list[0])

3332.0

In [37]:
print(room_list[0])
Cx, Cy = centroid(room_list[0])

{'name': 'Living Room', 'position': (12, 71), 'size': (68, 49)}
